<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-training-course/colab/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmtraining-colab} -->

# Training a 3B Llama on instruction dataset with Weights & Biases, HuggingFace, LoRA and Quantization

Tested on Google Colab V100 GPU. Check out [W&B HuggingFace documentation](https://docs.wandb.ai/guides/integrations/huggingface) for more details.

In [1]:
!conda env list

# conda environments:
#
base                     /anaconda
azureml_py310_sdkv2      /anaconda/envs/azureml_py310_sdkv2
azureml_py38             /anaconda/envs/azureml_py38
azureml_py38_PT_TF       /anaconda/envs/azureml_py38_PT_TF
finetune                 /anaconda/envs/finetune
jupyter_env              /anaconda/envs/jupyter_env



In [2]:
!python --version

Python 3.8.5


In [1]:
!python -m pip install -U wandb transformers trl datasets "protobuf==3.20.3" evaluate peft bitsandbytes accelerate sentencepiece -qqq

In [6]:
pip install -U wandb transformers trl datasets "protobuf==3.20.3" evaluate peft bitsandbytes accelerate sentencepiece -qqq

ERROR: azureml-dataset-runtime 1.51.0 has requirement pyarrow<=9.0.0,>=0.17.0, but you'll have pyarrow 15.0.0 which is incompatible.
ERROR: azureml-automl-dnn-nlp 1.51.0 has requirement datasets<=2.3.2,>=1.7.0, but you'll have datasets 2.18.0 which is incompatible.
ERROR: azureml-automl-dnn-nlp 1.51.0 has requirement transformers[sentencepiece]<=4.16.0, but you'll have transformers 4.38.2 which is incompatible.
ERROR: peft 0.9.0 has requirement torch>=1.13.0, but you'll have torch 1.12.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install trl

Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install protobuf

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install peft

     |████████████████████████████████| 755.5 MB 7.7 kB/s s eta 0:00:01
     |████████████████████████████████| 410.6 MB 5.9 kB/s s eta 0:00:01
     |████████████████████████████████| 99 kB 10.3 MB/s  eta 0:00:01
     |████████████████████████████████| 14.1 MB 102.5 MB/s eta 0:00:01
     |████████████████████████████████| 121.6 MB 62 kB/s s eta 0:00:01
     |████████████████████████████████| 56.5 MB 122.8 MB/s eta 0:00:01
     |████████████████████████████████| 731.7 MB 13 kB/s /s eta 0:00:01
     |████████████████████████████████| 23.7 MB 82.4 MB/s eta 0:00:01
     |████████████████████████████████| 167.9 MB 21 kB/s /s eta 0:00:01
     |████████████████████████████████| 166.0 MB 78 kB/s /s eta 0:00:01
     |████████████████████████████████| 196.0 MB 45 kB/s /s eta 0:00:01
     |████████████████████████████████| 823 kB 80.4 MB/s eta 0:00:01
     |████████████████████████████████| 124.2 MB 12 kB/s s eta 0:00:01
     |████████████████████████████████| 5.7 MB 118.3 MB/s eta 0:00:01
     |

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The Jupyter

In [3]:
%pip install bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [2]:
!wget https://github.com/wandb/edu/raw/main/llm-training-course/colab/utils.py

--2024-03-05 13:13:41--  https://github.com/wandb/edu/raw/main/llm-training-course/colab/utils.py
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wandb/edu/main/llm-training-course/colab/utils.py [following]
--2024-03-05 13:13:41--  https://raw.githubusercontent.com/wandb/edu/main/llm-training-course/colab/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8155 (8.0K) [text/plain]
Saving to: ‘utils.py.4’

utils.py.4          100%[===================>]   7.96K  --.-KB/s    in 0.008s  

2024-03-05 13:13:41 (1.03 MB/s) - ‘utils.py.4’ saved [8155/8155]



Let's grab the Alpaca (GPT-4 curated instructions and outputs) dataset:

In [3]:
import wandb
wandb.init(project="alpaca_ft", # the project I am working on
           job_type="train",
           tags=["hf_sft_lora", "3b"]) # the Hyperparameters I want to keep track of
artifact = wandb.use_artifact('capecape/alpaca_ft/alpaca_gpt4_splitted:v4', type='dataset')
artifact_dir = artifact.download()

wandb: Currently logged in as: balabala76. Use `wandb login --relogin` to force relogin


wandb:   2 of 2 files downloaded.  


In [4]:
from datasets import load_dataset
alpaca_ds = load_dataset("json", data_dir=artifact_dir)

In [5]:
# Let's subsample the training and test dataset - you may want to switch to full dataset in your experiments
alpaca_ds['train'] = alpaca_ds['train'].select(range(512))
alpaca_ds['test'] = alpaca_ds['test'].select(range(10))

Let's log the dataset also as a table so we can inspect it on the workspace.

In [6]:
def prompt_no_input(row):
    return ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:\n{output}").format_map(row)

def prompt_input(row):
    return ("Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n{output}").format_map(row)

def create_prompt(row):
    return prompt_no_input(row) if row["input"] == "" else prompt_input(row)

In [7]:
train_dataset = alpaca_ds["train"]
eval_dataset = alpaca_ds["test"]

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [9]:
from huggingface_hub import notebook_login
notebook_login()

In [44]:
#model_id = 'openlm-research/open_llama_3b_v2'
model_id = 'microsoft/phi-2'
#model_id = 'meta-llama/Llama-2-7b-hf'

Let's define our configurations for LoRA, quantization and model training so that it fits on our GPU.

In [32]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=64,  # the rank of the LoRA matrices
    lora_alpha=16, # the weight
    lora_dropout=0.1, # dropout to add to the LoRA layers
    bias="none", # add bias to the nn.Linear layers?
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj","v_proj","o_proj"], # the name of the layers to add LoRA
)

In [34]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [35]:
model_kwargs = dict(
    device_map={"" : 0},
    trust_remote_code=True,
    # low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    # use_flash_attention_2=True,
    use_cache=False,
    quantization_config=bnb_config,
)

In [36]:
from transformers import TrainingArguments
from trl import SFTTrainer

In [37]:
batch_size = 2
gradient_accumulation_steps = 16
num_train_epochs = 10

We'll add a `report_to="wandb"` flag here to get the benefits of [W&B HuggingFace integration](https://docs.wandb.ai/guides/integrations/huggingface).

In [38]:
output_dir = "./output/"
training_args = TrainingArguments(
    num_train_epochs=num_train_epochs,
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs=dict(use_reentrant=False),
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=1,
    save_strategy="epoch",
    report_to="wandb",
)

In [39]:
from utils import LLMSampleCB

trainer = SFTTrainer(
    model=model_id,
    model_init_kwargs=model_kwargs,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    packing=True,
    max_seq_length=1024,
    args=training_args,
    formatting_func=create_prompt,
    peft_config=peft_config,
)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:161: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [40]:
# remove answers
def create_prompt_no_anwer(row):
    row["output"] = ""
    return {"text": create_prompt(row)}

test_dataset = eval_dataset.map(create_prompt_no_anwer)

We will add a custom W&B callback to the trainer so that we can sample and log model generations in W&B dashboard. Review [W&B HuggingFace documentation](https://docs.wandb.ai/guides/integrations/huggingface) for the most up-to-date best practices.

In [41]:
wandb_callback = LLMSampleCB(trainer, test_dataset, num_samples=10, max_new_tokens=256)

In [42]:
trainer.add_callback(wandb_callback)

It's time to train!

In [43]:
trainer.train()
wandb.finish()

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
0,1.369800,1.366599
1,1.334900,1.241878
2,1.233100,1.155038
3,1.163800,1.111018
4,1.150700,1.096687
5,1.149600,1.095525


  0%|          | 0/10 [00:00<?, ?it/s]

Checkpoint destination directory ./output/checkpoint-1 already exists and is non-empty. Saving will proceed but saved results may be invalid.
wandb: Adding directory to artifact (./output/checkpoint-1)... Done. 10.0s
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

Checkpoint destination directory ./output/checkpoint-9 already exists and is non-empty. Saving will proceed but saved results may be invalid.
wandb: Adding directory to artifact (./output/checkpoint-9)... Done. 12.7s
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


  0%|          | 0/10 [00:00<?, ?it/s]

Checkpoint destination directory ./output/checkpoint-10 already exists and is non-empty. Saving will proceed but saved results may be invalid.
wandb: Adding directory to artifact (./output/checkpoint-10)... Done. 12.9s


eval/loss,██▅▅▃▃▁▁▁▁▁▁
eval/runtime,▃▃▄▄██▄▄▆▆▁▁
eval/samples_per_second,▆▆▅▅▁▁▆▆▃▃██
eval/steps_per_second,██▅▅▁▁▅▅▅▅██
train/epoch,▁▁▁▁▂▂▃▃▃▃▃▃▄▄▄▄▅▅▆▆▆▆▆▆▇▇▇▇██████
train/global_step,▁▁▁▁▁▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▆▆▆▆▆▆▆▇▇▇▇▇███████
train/grad_norm,▆▆▄▄▆▆██▆▆▄▄▃▃▂▂▂▂▁▁
train/learning_rate,████▇▇▆▆▅▅▄▄▃▃▂▂▁▁▁▁
train/loss,████▇▇▅▅▄▄▂▂▂▂▁▁▁▁▁▁
train/total_flos,▁▁
train/train_loss,▁▁


Check out the sample generations in your W&B workspace. We've trained on a very small sample of dataset, so likely they won't be good. Try to improve this result! Train on a larger dataset, experiment with different hyperparameters and settings. Then share a W&B report with your results. Good luck!